In [1]:
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config

# Introduction

This is just a quick set of tests to figure out how to compute the absorption of an individual layer from the power fluxes at the top and bottom of each layer

In [2]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim = setup_sim(sim)

In [17]:
fluxes = sim.get_fluxes()

DEBUG:nanowire.optics.simulate:Computing fluxes ...
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Substrate
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Air
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: ITO
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Finished computing fluxes!


In [4]:
print(fluxes)

{'ITO': ((889.3896940292223+0.40536162201509285j), (-31.320447816229574-0.40536162201509285j)), 'ITO_bottom': ((832.8167787694824-1.1020696710776718j), (-32.32968144724215+1.1020696710776718j)), 'Substrate_bottom': ((41.56120138174488+0j), 0j), 'NW_AlShell': ((805.4789693755682-1.018496961282608j), (-4.99187205333792+1.018496961282608j)), 'NW_SiO2_bottom': ((294.3421073335986-0.17843894992674514j), (-25.517551725745832+0.17843894992674514j)), 'Substrate': ((268.82455560785075+0j), 0j), 'NW_AlShell_bottom': ((288.4316133009568-1.6540491075088823j), (-15.301493749495123+1.6540491075088823j)), 'NW_SiO2': ((298.47308699452026-1.2484204892192103j), (-25.34296744306598+1.2484204892192103j)), 'Air': ((930.4485521295919+0j), (-72.37930591656783+0j)), 'Air_bottom': ((930.4485521295919+0j), (-72.37930591656784+0j))}


In [5]:
out_ito = fluxes['ITO_bottom'][0] + fluxes['ITO_bottom'][1]
in_nw = fluxes['NW_AlShell'][0] + fluxes['NW_AlShell'][1]
print(out_ito)
print(in_nw)
print(fluxes['ITO_bottom'])
print(fluxes['NW_AlShell'])

(800.4870973222403+0j)
(800.4870973222303+0j)
((832.8167787694824-1.1020696710776718j), (-32.32968144724215+1.1020696710776718j))
((805.4789693755682-1.018496961282608j), (-4.99187205333792+1.018496961282608j))


In [25]:
ito_inc = np.absolute(fluxes['ITO'][0])
ito_ref = np.absolute(fluxes['Air_bottom'][1])
ito_ref = np.absolute(fluxes['ITO'][1])
ito_trans = np.absolute(fluxes['NW_AlShell'][0])
print(ito_inc)
print(ito_ref)
print(ito_trans)
absorbed = ito_inc - ito_ref - ito_trans
print(absorbed)

889.389786406
31.3230708816
805.4796133
52.5871022242


In [8]:
total_incident_power = np.absolute(fluxes['Air'][0])
total_reflected_power = np.absolute(fluxes['Air'][1])
total_transmitted_power = np.absolute(sum(fluxes['Substrate_bottom']))
total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
print('Total Incident Power = {}'.format(total_incident_power))
print('Total Reflected Power = {}'.format(total_reflected_power))
print('Total Transmitted Power = {}'.format(total_transmitted_power))
print('Total Absorbed Power = {}'.format(total_absorbed_power))

Total Incident Power = 930.4485521295919
Total Reflected Power = 72.37930591656783
Total Transmitted Power = 41.56120138174488
Total Absorbed Power = 816.5080448312792


In [6]:
summed_absorbed_power = 0
for layer, (forw, back) in fluxes.items():
    if '_bottom' in layer:
        continue
    incident_power = np.absolute(forw)
    reflected_power = np.absolute(back)
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Incident Power: {}'.format(incident_power))
    print('Reflected Power: {}'.format(reflected_power))
    bottom = layer+'_bottom'
    transmitted_power = np.absolute(fluxes[bottom][0])
    bottom_reflected_power = np.absolute(fluxes[bottom][1])
    print('Transmitted Power: {}'.format(transmitted_power))
    print('Backward_bottom: {}'.format(bottom_reflected_power))
    #absorbed = incident_power + bottom_reflected_power - transmitted_power - reflected_power
    flux_top = np.absolute(forw+back)
    flux_bottom = np.absolute(fluxes[bottom][0]+fluxes[bottom][1]) 
    absorbed = flux_top - flux_bottom
    print('Absorbed in Layer: {}'.format(absorbed))
    summed_absorbed_power += absorbed
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))

-------------------------
Layer: ITO
Incident Power: 889.389786406072
Reflected Power: 31.323070881600398
Transmitted Power: 832.8175079556967
Backward_bottom: 32.34845993304879
Absorbed in Layer: 57.582148890752364
-------------------------
Layer: Substrate
Incident Power: 268.82455560785075
Reflected Power: 0.0
Transmitted Power: 41.56120138174488
Backward_bottom: 0.0
Absorbed in Layer: 227.2633542261059
-------------------------
Layer: NW_SiO2
Incident Power: 298.4756978612436
Reflected Power: 25.373698046169867
Transmitted Power: 294.342161421096
Backward_bottom: 25.518175611414016
Absorbed in Layer: 4.3055639436014985
-------------------------
Layer: Air
Incident Power: 930.4485521295919
Reflected Power: 72.37930591656783
Transmitted Power: 930.4485521295919
Backward_bottom: 72.37930591656784
Absorbed in Layer: 1.1368683772161603e-13
-------------------------
Layer: NW_AlShell
Incident Power: 805.4796133002919
Reflected Power: 5.094715169372871
Transmitted Power: 288.4363559432873

So if the loop above computed things properly, then summed absorbed power should be equal to the total absorbed power

In [9]:
diff = total_absorbed_power - summed_absorbed_power
print('Diff = {}'.format(diff))

Diff = 5.070432962384075e-11


So they appear to be pretty close. This device didn't have an air layer below everything. I try adding an air layer below to
see if that improves the agreement between these two methods

In [11]:
below_conf = Config('AbsorptionTest_airbelow.yml')
below_sim = Simulator(below_conf)
below_sim = setup_sim(below_sim)

DEBUG:nanowire.optics.simulate:Logger initialized
DEBUG:nanowire.optics.simulate:Building layer: Air
DEBUG:nanowire.optics.simulate:Layer Order 1
DEBUG:nanowire.optics.simulate:Building layer: ITO
DEBUG:nanowire.optics.simulate:Layer Order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Layer Order 3
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Building object shell of type circle at order 1
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Layer Order 4
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Building object shell of type circle at order 1
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: Substrate
DEBUG:nanowire.optics.simulate:Layer Order 5
DE

In [12]:
fluxes = below_sim.get_fluxes()

DEBUG:nanowire.optics.simulate:Computing fluxes ...
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Substrate
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: ITO
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Air
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Bottom_Air
DEBUG:nanowire.optics.simulate:Finished computing fluxes!


In [13]:
print(fluxes)

{'Substrate_bottom': ((43.60219291195291+0.6932449963104814j), (-15.07239896981021-0.6932449963104814j)), 'Bottom_Air_bottom': ((28.52979394214275+0j), 0j), 'NW_AlShell': ((814.5027719852595-0.797432675361371j), (-2.617026481935543+0.797432675361371j)), 'Bottom_Air': ((28.52979394214275+0j), 0j), 'NW_SiO2': ((299.1838150595673-1.7439443101523118j), (-14.872676558269914+1.7439443101523118j)), 'Substrate': ((282.21592462896365+0.5540592160286817j), (-2.7521874656798158-0.5540592160286817j)), 'NW_SiO2_bottom': ((294.8071086249769-0.620316201855837j), (-15.34337146169093+0.620316201855837j)), 'ITO': ((896.507349202051+0.5037645865747891j), (-26.29738076318455-0.5037645865747891j)), 'Air': ((930.4485521295919+0j), (-60.23858369069318+0j)), 'ITO_bottom': ((839.1481953357867-1.0767936010322146j), (-27.262449832452976+1.0767936010322146j)), 'Air_bottom': ((930.4485521295919+0j), (-60.23858369069321+0j)), 'NW_AlShell_bottom': ((291.9930683146283-1.3588527120678293j), (-7.681929813325565+1.35885

In [14]:
total_incident_power = np.absolute(fluxes['Air'][0])
total_reflected_power = np.absolute(fluxes['Air'][1])
total_transmitted_power = np.absolute(fluxes['Bottom_Air_bottom'][0])
total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
print('Total Incident Power = {}'.format(total_incident_power))
print('Total Reflected Power = {}'.format(total_reflected_power))
print('Total Transmitted Power = {}'.format(total_transmitted_power))
print('Total Absorbed Power = {}'.format(total_absorbed_power))

Total Incident Power = 930.4485521295919
Total Reflected Power = 60.23858369069318
Total Transmitted Power = 28.52979394214275
Total Absorbed Power = 841.680174496756


In [15]:
summed_absorbed_power = 0
for layer, (forw, back) in fluxes.items():
    if '_bottom' in layer:
        continue
    incident_power = np.absolute(forw)
    reflected_power = np.absolute(back)
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Incident Power: {}'.format(incident_power))
    print('Reflected Power: {}'.format(reflected_power))
    bottom = layer+'_bottom'
    transmitted_power = np.absolute(fluxes[bottom][0])
    bottom_reflected_power = np.absolute(fluxes[bottom][1])
    print('Transmitted Power: {}'.format(transmitted_power))
    print('Backward_bottom: {}'.format(bottom_reflected_power))
    #absorbed = incident_power + bottom_reflected_power - transmitted_power - reflected_power
    flux_top = np.absolute(forw+back)
    flux_bottom = np.absolute(fluxes[bottom][0]+fluxes[bottom][1]) 
    absorbed = flux_top - flux_bottom
    print('Absorbed in Layer: {}'.format(absorbed))
    summed_absorbed_power += absorbed
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))

-------------------------
Layer: NW_AlShell
Incident Power: 814.5031623453302
Reflected Power: 2.735822815696572
Transmitted Power: 291.9962301545763
Backward_bottom: 7.801187496141485
Absorbed in Layer: 527.5746070020211
-------------------------
Layer: Bottom_Air
Incident Power: 28.52979394214275
Reflected Power: 0.0
Transmitted Power: 28.52979394214275
Backward_bottom: 0.0
Absorbed in Layer: 0.0
-------------------------
Layer: NW_SiO2
Incident Power: 299.18889774748374
Reflected Power: 14.974573441798736
Transmitted Power: 294.80776124113356
Backward_bottom: 15.35590570437695
Absorbed in Layer: 4.847401338011423
-------------------------
Layer: Substrate
Incident Power: 282.21646850564156
Reflected Power: 2.8074040430816876
Transmitted Power: 43.607703623970686
Backward_bottom: 15.088333219081736
Absorbed in Layer: 250.93394322114113
-------------------------
Layer: ITO
Incident Power: 896.5074907395069
Reflected Power: 26.302205492365
Transmitted Power: 839.148886205402
Backward_b

In [16]:
diff = total_absorbed_power - summed_absorbed_power
print('Diff = {}'.format(diff))

Diff = 4.945377440890297e-11
